# CS202: Compiler Construction

## In-class Exercises, Week of 02/27/2023

----

# RCO

## Question 1

How do we handle `if` statements in rco?

If is not an operator; its arguments don't need to be atomic.

- call rco_exp on the condition
- call rco_stmts on the then and else statements

----

# Explicate-Control

## Question 2

Describe the idea behind the explicate-control pass.

The pass is designed to compile if statements into a control flow graph (gotos/jumps); not structural if statements.

The output is in a intermediate language called `Cif`.

- Similar to Python in most respects (expressions, statements)
- Difference
    - If statements must have gotos in then and else branches
    - Program is represented by a dict mapping labels to list of statements (a control flow graph)

Four pieces to consider when compiling an if statement:
- the condition
- the then branch (statements)
- the else branch (statements)
- the rest of the program (statements)
  - this is called the continuation

We will process the statements backwards, so the continuation is the first thing we process.

## Question 3

Compile this program into a $C_{if}$ program, describing the input and output of explicate-control.

```
if 5 == 6:
    print(1)
else:
    print(0)
```

```
label_2: 
  return 0
label_3:
  print(1)
  goto label_2
label_4:
  print(0)
  goto label_2
start: 
  tmp_1 eq(5,6)
  if tmp_1: goto label_3 else: goto label_4
```

## Question 4

Describe the structure of `explicate_control`.

Start by following the grammar 

- `ec_atml` this compiles an atomic expression into a Cif atom
  - Constants -> cif.Constant
  - Variables -> cif.Variable
- `ec_expr` this compiles an expression into a Cif expression
  - Prim(op, args) -> cif.Prim(op, args)
  - else call `ec_atml`
- `ec_stmt` takes a stmt and a continutation, returns a list of Cif statements
  - Assign(x, e) => [cif.Assign(x, ec_expr(e))] + cont
  - Print(e) => [cif.Print(ec_expr(e))] + cont
  - If(condition, then_stmts, else_stmts) => 
    - cond_label = create a block for cont
    - then_label = create a block for ec_stmts(then_stmts,[cif.Goto(cond_label)])
    - else_label = create a block for ec_stmts(else_stmts,[cif.Goto(cond_label)])
    - return [cif.If(ec_expr(condition), cif.Goto(then_label), cif.Goto(else_label))] + cont
- `ec_stmts` takes a list of stmts and a continuation, returns a list of Cif statements
    - process the list of the statements in reverse
    - update "current continuation" by calling ec_stmt on each stmt and setting the continuation to whatever comes back
    - start with the continuation: [cif.Return(0)]
    - call ec_stmts on the statements of the program
    - set basic_blocks ['start'] to the result
- pass should have a "global" basic_blocks: Dict[str, List[cif.Stmt]]

- pass should jave a create_block function that adds a new block to basic_blocks with a unique name (using gensym)

----

# Select Instructions

## Question 5

Describe the new cases in `select_instructions`.

New cases:

Textbook section 5.8 for details.

- `Assign(x, Prim(op, [a1, a2]))` and `op` is a comparison primitive:
    - cmpq a2,a1 (compares a1 to a2, sets Eflags register; make sure that a1 and a2 are ordered 'backwards')
    - set[cc] %al (sets bytes register %a1 to the value of EFLAGS)
        - cc is the condition code for the desired comparison op
        - eq(==): e, gt(>): g, gte(>=): ge, lt(<): l, lte(<=): le
    - `movzbq %al, #x` (moves byte into a 64-int location)
- `If(condition, Goto(label1), Goto(label2))`
    - cmpq condition, $1
    - je label1
    - jmp label2
- `Assign(x, Prim('not', [a1]))`
    - movq a1, #x
    - xorq $1, #x
- `Prim('not', [a1])`
- `Return(a1)`
    - movq a1, %rax
    - jmp conclusion
- `Goto(label)` => `jmp label`

----

# Register Allocation

## Question 6

Describe the liveness analysis for a control-flow graph.

- Challenge: now I have many blocks, and jumps between them; a variable might be read in a block I am jumping to, so its better remain live "across" the jump

- Means I can't consider each block in isolation, I need to look at all blocks at once
- Treat things that might get read by the block I'm jumping to as things 

## Question 7

Here is an x86 assembly program. Compute its live-after sets.

```
start:
  movq $42, #x
  cmpq $6, $5
  je label_2
  jmp label_3
label_2:
  addq $3, #x
  jmp conclusion
label_3:
  addq $4, #x
  jmp conclusion
```

When I find a jump instruction, computer the live-after set for the target, and the live-before set for the target, and treat the live-before set as the variables read by the jump.

the `conclusion` block always has the empty live-after set

```
start:          {}
  movq $42, #x  {x}
  cmpq $6, $5   {x}
  je label_2    {x}
  jmp label_3   {}
label_2:          {x} live-before set for label_2
  addq $3, #x     {}
  jmp conclusion  {}
label_3:          {x} live-before set for label_3
  addq $4, #x     {}
  jmp conclusion  {}
```

- Global dict that remember the list of live-before sets for each block
- Global dict that remembers the list of live-after sets for each block

## Question 8

Describe the change to building the interference graph.

No change

## Question 9

Describe the change to graph coloring and assigning homes.

No change

----

# Patch Instructions

## Question 10

Describe the changes to patch_instructions.

cmpq instruction is now a special case

----

# Prelude & Conclusion


## Question 11

Describe the changes to prelude_and_conclusion.

Prelude goes in main
Conclusion code goes in conclusion